In [1]:
from src.utils import globals as GLOBALS
import pandas as pd
import sys
from os import listdir
pd.set_option('display.max_columns', 500)

def createDirs(liga):
    #main_path = "C:/Users/Robert/Documents/Projekte/dev/bettingTool/data/leagues/" + liga
    main_path = GLOBALS.DATA_FOLDER+"/leagues/" + liga
    newpathMarktwert = main_path + "/marktwert"
    newpathSeason = main_path + "/season"
    newpathxG = main_path + "/xG"
    newpathLineup = main_path + "/lineup"

def downloadLeague(liga="D1", season="1920"):
    if liga in GLOBALS.MAIN_LEAGUES:
        url = 'https://www.football-data.co.uk/mmz4281/' + season + '/' + liga + '.csv'
    elif GLOBALS.EXTRA_LEAGUES:
        url = f'https://www.football-data.co.uk/new/{liga}.csv'

    data = pd.read_csv(url, sep=",", encoding='cp1252', error_bad_lines=False)  # use sep="," for coma separation. 
    main_path =GLOBALS.DATA_FOLDER+"/leagues/"
    data.to_csv(main_path + liga + '/season/' + liga + '_' + season + '.csv')
    return(data)

def split_date(df):
    if 'Date' in df.columns:
        new_date = (df["Date"].str.split("/",expand=True))
        new_date.columns = ["day","month","year"]
        new_date = new_date.astype('int')

        new_date.year[new_date.year < 2000] = new_date.year[new_date.year < 2000]+2000

        df = pd.concat([df,new_date],axis=1)
    else:
        print('Keine Spalte "Date" in Spalten vorhanden')
    return df

Temporary folder exists. Deleting and recreating...
Output folder exists. Deleting and recreating...


Lade alle Ligen herunter

In [2]:
#Ini
SEASONS = ["1213","1314","1415","1516","1617","1718","1819","1920","2021","2122","2223"]
#Update
SEASONS = ["2223"]

train_leagues = ["D1", "F1", "I1", "E0", "E1","E2","E3", "SP1", "D2", "F2", "SP2", "I2", "N1", "SC0", "B1", "P1", "T1", "G1"]
# Iterate through leagues
for l in train_leagues:
    # create directories
    createDirs(l)
    # Download seasons for every league
    for s in SEASONS:
        try:
            downloadLeague(liga=l, season = s)
        except Exception as e:
            print("Unexpected error: {e}}")
            print(f'Liga: {l} Season: {s}')
            pass

C:\Users\Robert\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:14: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  


Speichere alle Rohdaten in einer Datei


In [3]:
import os 
leagues_folder = GLOBALS.DATA_FOLDER+"/leagues/"
leagues = listdir(leagues_folder)
data = pd.DataFrame(columns = list(set(GLOBALS.ATTRIBUTES_MAIN_LEAGUE+GLOBALS.ATTRIBITES_EXTRA_LEAGUE)))
import pandas as pd
for l in leagues:
    if os.path.exists(f"{leagues_folder}/{l}/season/"):
        seasons = listdir(f"{leagues_folder}/{l}/season/")
        for s in seasons:
 
            df = pd.read_csv(f"{leagues_folder}/{l}/season/{s}",index_col=0)
            data = pd.concat([df,data],axis=0)


data = data.drop([i for i in data.columns if 'unnamed' in i.lower()],axis=1)


Speichere Daten einzeln ab für Extra und Main Leagues

In [4]:
import pickle

# filename_main = GLOBALS.DATA_FOLDER+'raw_database.sav'
# pickle.dump(data, open(filename_main, 'wb'))

filename_main = GLOBALS.DATA_FOLDER+'main_leagues_database_raw.sav'
pickle.dump(data[~data.HomeTeam.isna()][GLOBALS.ATTRIBUTES_MAIN_LEAGUE], open(filename_main, 'wb'))

filename_extra = GLOBALS.DATA_FOLDER+'extra_leagues_database_raw.sav'
pickle.dump(data[data.HomeTeam.isna()][GLOBALS.ATTRIBITES_EXTRA_LEAGUE], open(filename_extra, 'wb'))

Lade Daten ein

In [5]:
df = pickle.load(open(filename_main, 'rb'))

Restrukturiere die Daten und Mapping

In [ ]:
filename_main = GLOBALS.DATA_FOLDER+'main_leagues_database_raw.sav'

def load_team_opponent(filename_main:str):
    df = pickle.load(open(filename_main, 'rb'))
    mapping = pd.read_csv(GLOBALS.SETTINGS_FOLDER+'mapping_team_opponent.csv',"\t")


    map_home = dict()
    map_away=dict()
    for i in mapping.columns:
        if mapping[i][0]!="x":
            map_home[i] = mapping[i][0]
        if mapping[i][1]!="x":
            map_away[i] = mapping[i][1]

    df_home = df[[i for i in df.columns if i in  map_home]].rename(map_home,axis=1)
    df_home['atHome']=True
    #df_home.index = df_home.index+'__home' 
    df_away = df[[i for i in df.columns if i in  map_away]].rename(map_away,axis=1)
    df_away['atHome']=False
    #df_away.index = df_away.index+'__away' 

    df_team_opponent = pd.concat([df_home,df_away],axis=0)
    return df_team_opponent